In [2]:
from typing import List

import numpy as np
import scipy.sparse as sp

# Spectral clustering users based on their preferences

The goal of this task is to find groups of users with similar preferences using **Spectral clustering**. 
Given a fragment of the Yelp social network, represented by an undirected weighted graph.
Nodes in the graph represent users.
If two users are connected by an edge of weight $w$, it means that they have both left positive reviews to the same $w$ restaurants.

Additionally, given a matrix `F` that encodes user preferences to different categories of restaurants. If `F[i, c] = 1`, then user `i` likes restaurants in category `c`.

## Load the data

* `N` = number of users (nodes in the graph)
* `C` = number of categories
* The graph is stored as a _sparse adjacency matrix_ `A` (shape `[N, N]`).
* User preferences are stored in a _feature matrix_ `F` (shape `[N, C]`). They will only be used for the final part of the assignment (Part 3)
* Name of each category is provided in the list `categories` (length `[C]`).

In [3]:
A = sp.load_npz('A.npz')
F = np.load('F.npy')
categories = np.load('categories.npy', allow_pickle=True).tolist()

In [4]:
assert A.shape[0] == F.shape[0]
assert F.shape[1] == len(categories)

In [5]:
print(f'The adjacency matrix is {"symmetric" if (A != A.T).sum() == 0 else "asymmetric"}')

The adjacency matrix is symmetric


# 1. Implementing spectral clustering
## 1.1. Construct the graph Laplacian

First, we need to construct the Laplacian for the given graph (*Do only use sparse operations, see [Scipy Sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)*). 

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

If our goal is to minimize the **ratio cut**, we will need to use the **unnormalized Laplacian**, defined as
$$L_{unnorm} = D - A.$$

If our goal is to minimze the **normalized cut**, we will need to use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

In [6]:
def construct_laplacian(A: sp.csr_matrix, norm_laplacian: bool) -> sp.csr_matrix:
    """Construct Laplacian of a graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    norm_laplacian : bool
        Whether to construct the normalized graph Laplacian or not.
        If True, construct the normalized (symmetrized) Laplacian, L = I - D^{-1/2} A D^{-1/2}.
        If False, construct the unnormalized Laplacian, L = D - A.
        
    Returns
    -------
    L : scipy.sparse.csr_matrix, shape [N, N]
        Laplacian of the graph.
        
    """
 
    N = A.shape[0]
    D = A.sum(axis=1)
    D = 1/np.sqrt(D)
    D_sqrt = np.diag(D.flat)
    D_sqrt = sp.csc.csc_matrix(D_sqrt)
    
    if norm_laplacian:
        I = np.eye(N)
        L = D_sqrt@A
        L = L@D_sqrt
        L = I - L
    else:
        #D = np.diag(A.sum(axis=1).flat)
        D = A.sum(axis=1)
        D = np.diag(D.flat)

        L = D - A
    return L

## 1.2. Spectral embedding

Now, we have to compute the spectral embedding for the given graph.

In order to partition the graph into $k$ clusters, such that the desired cut (ratio or normalized) is minimized, we need to consider the $k$ eigenvectors corresponding to the $k$ smallest eigenvalues of the graph Laplacian.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).

Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [7]:
from scipy.sparse.linalg import eigsh

In [8]:
help(eigsh)

Help on function eigsh in module scipy.sparse.linalg.eigen.arpack.arpack:

eigsh(A, k=6, M=None, sigma=None, which='LM', v0=None, ncv=None, maxiter=None, tol=0, return_eigenvectors=True, Minv=None, OPinv=None, mode='normal')
    Find k eigenvalues and eigenvectors of the real symmetric square matrix
    or complex Hermitian matrix A.
    
    Solves ``A * x[i] = w[i] * x[i]``, the standard eigenvalue problem for
    w[i] eigenvalues with corresponding eigenvectors x[i].
    
    If M is specified, solves ``A * x[i] = w[i] * M * x[i]``, the
    generalized eigenvalue problem for w[i] eigenvalues
    with corresponding eigenvectors x[i].
    
    Note that there is no specialized routine for the case when A is a complex
    Hermitian matrix. In this case, ``eigsh()`` will call ``eigs()`` and return the
    real parts of the eigenvalues thus obtained.
    
    Parameters
    ----------
    A : ndarray, sparse matrix or LinearOperator
        A square operator representing the operation ``

In [9]:
def spectral_embedding(A: sp.csr_matrix, num_clusters: int, norm_laplacian: bool) -> np.array:
    """Compute spectral embedding of nodes in the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    embedding : np.array, shape [N, num_clusters]
        Spectral embedding for the given graph.
        Each row represents the spectral embedding of a given node.
    
    """
    if (A != A.T).sum() != 0:
        raise ValueError("Spectral embedding doesn't work if the adjacency matrix is not symmetric.")
    if num_clusters < 2:
        raise ValueError("The clustering requires at least two clusters.")
    if num_clusters > A.shape[0]:
        raise ValueError(f"We can have at most {A.shape[0]} clusters (number of nodes).")
   
    laplacian = construct_laplacian(A, norm_laplacian)
    eigenvalues, eigenvectors = eigsh(laplacian, num_clusters, which='SA')

    return eigenvectors

## 1.3. Determine the clusters based on the spectral embedding

Use the K-means algorithm for assigning nodes to clusters, once the spectral embedding is computed.

When using the **normalized Laplacian**, the rows of the embedding matrix **have to** be normalized to have unit $L_2$ norm. The justification of this approach is not trivial.

In [10]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [11]:
def spectral_clustering(A: sp.csr_matrix, num_clusters: int, norm_laplacian: bool, seed: int = 42) -> np.array:
    """Perform spectral clustering on the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
    seed : int, default 42
        Random seed to use for the `KMeans` clustering.
        
    Returns
    -------
    z_pred : np.array, shape [N]
        Predicted cluster indicators for each node.
        
    """
    model = KMeans(num_clusters, random_state=seed)
   
    embedding = spectral_embedding(A=A, num_clusters=num_clusters, norm_laplacian=norm_laplacian)
    embedding = normalize(embedding)
    model.fit(embedding)
    z_pred = model.predict(embedding)
    
    return z_pred

# 2. Quantitatively evaluate the results

In [12]:
def labels_to_list_of_clusters(z: np.array) -> List[List[int]]:
    """Convert predicted label vector to a list of clusters in the graph.
    This function is already implemented, nothing to do here.
    
    Parameters
    ----------
    z : np.array, shape [N]
        Predicted labels.
        
    Returns
    -------
    list_of_clusters : list of lists
        Each list contains ids of nodes that belong to the same cluster.
        Each node may appear in one and only one partition.
    
    Examples
    --------
    >>> z = np.array([0, 0, 1, 1, 0])
    >>> labels_to_list_of_clusters(z)
    [[0, 1, 4], [2, 3]]
    
    """
    return [np.where(z == c)[0] for c in np.unique(z)]

## 2.1. Compute ratio cut

In [13]:
def compute_ratio_cut(A: sp.csr_matrix, z: np.array) -> float:
    """Compute the ratio cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    ratio_cut : float
        Value of the cut for the given partition of the graph.
        
    """
    
    clusters = labels_to_list_of_clusters(z)
    ratio_cut = 0
    N = A.shape[0]
    
    for c in clusters:
        ratios = [i for i in range(N)]
        n_c = len(c)
        sum_weight = 0
        for ele in c:
            ratios.remove(ele)
            
        ratios_not_c = ratios
        row = A[c,:]
        row = row[:,ratios_not_c]
        sum_weight = row.sum()
        ratio_cut = ratio_cut + sum_weight/n_c  


    return ratio_cut

## 2.2. Compute normalized cut

**Important**: if a cluster only contains a single node, define its volume to be 1 to avoid division by zero errors.

In [14]:
def compute_normalized_cut(A: sp.csr_matrix, z: np.array) -> float:
    """Compute the normalized cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    norm_cut : float
        Value of the normalized cut for the given partition of the graph.
        
    """
    
    clusters = labels_to_list_of_clusters(z)
    norm_cut = 0
    N = A.shape[0]
    for c in clusters:
        C = 0
        ratios = [i for i in range(N)]
        n_c = len(c)
        sum_weight = 0
        for ele in c:
            ratios.remove(ele)
            C = C + A[ele,:].sum()
        
        ratios_not_c = ratios
        row = A[c,:]
        row = row[:,ratios_not_c]
        sum_weight = row.sum()
        norm_cut = norm_cut + sum_weight/C  

    return norm_cut

Notice, how using the unnormalized Laplacian leads to a much better ratio cut, while the normalized Laplacian leads to better normalized cut.

In [15]:
num_clusters = 6

In [16]:
np.random.seed(12903)
norm_laplacian = False
z_unnorm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_unnorm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_unnorm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_unnorm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_unnorm)]))

When using L_unnorm:
 ratio cut = 806.161
 normalized cut = 4.943
 sizes of partitions are: [31, 3344, 4, 1, 2, 2]


In [17]:
np.random.seed(12323)
norm_laplacian = True
z_norm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_norm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_norm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_norm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_norm)]))

When using L_norm:
 ratio cut = 5942.851
 normalized cut = 4.343
 sizes of partitions are: [742, 577, 754, 572, 350, 389]


# 3. Visualize the results

Print out the 5 most popular types of restaurants visited by the users in each cluster.

In [18]:
def print_top_categories_for_each_cluster(top_k: int, z: np.array, F: sp.csr_matrix, categories: List[str]):
    """Print the top-K categories among users in each cluster.
    For each cluster, the function prints names of the top-K categories,
    and number of users that like the respective category (separated by a comma).
    The function doesn't return anything, just prints the output.
    
    Parameters
    ----------
    top_k : int
        Number of most popular categories to print for each cluster.
    z : np.array, shape [N]
        Cluster labels.
    F : sp.csr_matrix, shape [N, C]
        Matrix that tells preferences of each user to each category.
        F[i, c] = 1 if user i gave at least one positive review to at least one restaurant in category c.
    categories : list, shape [C]
        Names of the categories.
        
    """
    n_categories = len(categories)

    clusters = labels_to_list_of_clusters(z)
    for index, cluster in enumerate(clusters):
        print("Most popular categories in cluster " + str(index))
        preferences = np.zeros(n_categories)
        for point in cluster:
            preferences += F[point, :]
        top_results = preferences.argsort()[-top_k:][::-1]
        for result in top_results:
            print(" - " + categories[result])
        print('\n')




In [19]:
np.argsort([333,5,10])

array([1, 2, 0])

In [20]:
np.random.seed(23142)
z_norm = spectral_clustering(A, num_clusters, True)
r = print_top_categories_for_each_cluster(5, z_norm, F, categories)

Most popular categories in cluster 0
 - Breakfast & Brunch
 - Sandwiches
 - Italian
 - Pizza
 - Coffee & Tea


Most popular categories in cluster 1
 - Japanese
 - Chinese
 - Asian Fusion
 - Sushi Bars
 - Desserts


Most popular categories in cluster 2
 - Breakfast & Brunch
 - Italian
 - American (Traditional)
 - Sandwiches
 - Pizza


Most popular categories in cluster 3
 - Japanese
 - Breakfast & Brunch
 - Sandwiches
 - Italian
 - Asian Fusion


Most popular categories in cluster 4
 - Seafood
 - Mexican
 - Sandwiches
 - Japanese
 - Breakfast & Brunch


Most popular categories in cluster 5
 - Specialty Food
 - Thai
 - Breakfast & Brunch
 - Japanese
 - Ethnic Food


